<a href="https://colab.research.google.com/github/greyhound101/shopee/blob/main/ocr_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import load_model, model_from_json
RESIZE_FACTOR = 2
import tensorflow as tf
json_file = open('../input/east-keras/model.json')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, custom_objects={'tf': tf.compat.v1, 'RESIZE_FACTOR': RESIZE_FACTOR})
model.load_weights('../input/east-keras/EAST_IC1513_model.h5')

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=512, batch_size=32, path=''): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)/255
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

In [ ]:


model.summary()



In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
def mean(x):
    x = K.mean(x, axis=-1, keepdims=True)
    return x

out=model.get_layer('pred_geo_map').output
output=Lambda(mean,name='efi')(out)
mod=Model(inputs=model.inputs,outputs=output)

mod.summary()



In [ ]:


import cv2
import time
import math
import os
import argparse
import numpy as np
import tensorflow as tf
from keras.models import load_model, model_from_json

import glob
img_list=glob.glob('../input/shopee-product-matching/train_images/*')

import pandas as pd
test = pd.read_csv('../input/shopee-product-matching/train.csv')

import gc
BASE = '../input/shopee-product-matching/train_images/'

# WGT = '../input/shopee-efficientnetb6-arcmarginproduct/EfficientNetB6_512_42.h5'
# model = efn.EfficientNetB6(weights=WGT,include_top=False, pooling='avg', input_shape=None)

A = []
B = []
CHUNK = 1024*4

print('Computing image embeddings...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for i,j in enumerate( range( CTS ) ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    test_gen = DataGenerator(test.iloc[a:b], batch_size=16, path=BASE)
    a = mod.predict(test_gen,verbose=1,use_multiprocessing=True, workers=4)
    A.append(a)
#     B.append(b)
    del test_gen
    _ = gc.collect()
del mod
_ = gc.collect()
image_embeddings = np.concatenate(A)
# image_embeddings1 = np.concatenate(B)
print('image embeddings shape',image_embeddings.shape)



In [ ]:
np.save('features.npy',image_embeddings)